### Notes


1. Supported formats for testing image are jpg, jpeg, png
2. The labels file should have txt format
3. The testing image and it's label must have the same name (for obvious reasons the files formats will be different)
4. To test different model, algorithm just change the variable ALGORITHM to call your function (the function need to be imported)
5. Function should return a list of tuples which contains 'detected_class' and 'coordinates'. The 'detected_class' must be a name of detected object stored as string. The 'coordinates' must be a list prepared according to this scheme: [x_center, y_center, width, height].

### Importing libraries

In [1]:
from collections import namedtuple
from typing import List, Dict, Tuple
from ultralytics import YOLO
import cv2
import glob
import time
import numpy as np
import matplotlib.pyplot as plt
import os

### Loading test images

In [2]:
supported_formats = ["jpg", "jpeg", "png"]

TestImagePair = namedtuple("TestImage", ["name", "image", "labels", "shapes"])
test_images: List[TestImagePair] = []

for file_format in supported_formats:
    files = glob.glob(os.path.join("test_images/images", "*." + file_format))
    for file in files:
        label_filepath = file.replace("images", "labels").replace(file_format, "txt").replace("labels", "images", 1)

        image = cv2.imread(file, cv2.IMREAD_COLOR)
        labels = []
        name = os.path.basename(file)

        with open(label_filepath, "r") as label_file:
            labels = label_file.read().splitlines()

        test_images.append(TestImagePair(name, image, labels, len(labels)))

print(f"Loaded {len(test_images)} test images")

Loaded 7 test images


### Evaluating

In [7]:
def test_yolo_model(image):
    model = YOLO("YoloV8-finetune/DroneModel/yolov8n_V2_320/weights/best_ncnn_model", task='detect')
    results = model(image, verbose=False)

    temp = []
    img_width, img_height, _ = image.shape

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x, y, width, height = box.xywh[0].tolist()
            detected_class = int(box.cls[0].item())
            x /= img_width
            y /= img_height
            width /= img_width
            height /= img_height

            temp.append((detected_class, [x, y, width, height]))

    return temp

In [9]:
ALGORITHM = test_yolo_model
TOLERANCE = 10 # 5%

detected_good = 0
all_labels_cnt = 0

# TODO add counter when label is not detected or when model detects to many labels
for timage in test_images:
    results: List[Tuple[str, List]] = ALGORITHM(timage.image)

    all_labels_cnt += len(timage.labels)

    if len(results) > timage.shapes:
        continue

    image_width, image_height, _ = timage.image.shape

    for i, item in enumerate(results):
        labels_split = timage.labels[i].split(" ")
        detected_class = labels_split[0]
        pred_coordinates = labels_split[1:]
        coordinates = results[i][1]

        if results[i][0] != detected_class:
            continue

        x_diff = np.abs(int(coordinates[0]) / image_width - int(pred_coordinates[0]) / image_width)
        y_diff = np.abs(int(coordinates[1]) / image_height - int(pred_coordinates[1]) / image_height)
        width_diff = np.abs(int(coordinates[2]) / image_width - int(pred_coordinates[2]) / image_width)
        height_diff = np.abs(int(coordinates[3]) / image_height - int(pred_coordinates[3]) / image_height)

        if np.any([x for x in [x_diff, y_diff, width_diff, height_diff] if x > (TOLERANCE / 100)]):
            continue

        detected_good += 1

try:
    print(f"Number of correct detections: {detected_good}")
    print(f"Number of wrong object detections: {all_labels_cnt - detected_good}")
    print(f"All labels: {all_labels_cnt}")
    print(f"Percentage of correct object detections: {detected_good / all_labels_cnt * 100:.2f}%")

except ZeroDivisionError:
    print(f"Error loading labels\n")

Loading /home/bartlomiej/Documents/JetBrains/PycharmProjects/ShapesRecogintion_RPI5/YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_ncnn_model for NCNN inference...
Loading /home/bartlomiej/Documents/JetBrains/PycharmProjects/ShapesRecogintion_RPI5/YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_ncnn_model for NCNN inference...
Loading /home/bartlomiej/Documents/JetBrains/PycharmProjects/ShapesRecogintion_RPI5/YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_ncnn_model for NCNN inference...
Loading /home/bartlomiej/Documents/JetBrains/PycharmProjects/ShapesRecogintion_RPI5/YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_ncnn_model for NCNN inference...
Loading /home/bartlomiej/Documents/JetBrains/PycharmProjects/ShapesRecogintion_RPI5/YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_ncnn_model for NCNN inference...
Loading /home/bartlomiej/Documents/JetBrains/PycharmProjects/ShapesRecogintion_RPI5/YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_n

In [13]:
model = YOLO("YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_ncnn_model", task='detect')
result = model("test_images/images/shapes_eval_1.jpg", verbose=False)

result[0].show()

Loading /home/bartlomiej/Documents/JetBrains/PycharmProjects/ShapesRecogintion_RPI5/YoloV8-finetune/DroneModel/yolov8n_V2_640/weights/best_ncnn_model for NCNN inference...
